<a href="https://colab.research.google.com/github/mehtamohit013/comms_lidar_ML/blob/main/BS_Selection/TF_5bs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab 

In [ ]:
!mkdir tb_logs
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%%capture _
!unzip /gdrive/MyDrive/BTP/5_bs/5_bs_ml.zip
# !unzip /gdrive/MyDrive/BTP/Webots_data/val_data.zip
!pip install pytorch-lightning

In [ ]:
!pip install  --upgrade pandas

# Imports

In [ ]:
%load_ext tensorboard

In [12]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from enum import Enum

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook
import os
import matplotlib.pyplot as plt
import random
import scipy.io as io
import math
import getpass
import time
from typing import Union

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

seed = 0

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pl.seed_everything(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True #Use only while training, not while deterministic 

Global seed set to 0


0

# Data and Hyper Params

## Hyper Params

In [13]:
user = getpass.getuser()

HOME = os.environ['HOME']

if user == 'root':  #For google colab
    dpath = './chicago'
    save_dir = './model_state_dict'
else:
    dpath = os.environ['HOME'] + '/webots_code/data/chicago'
    save_dir = os.path.join(os.environ['HOME'],'webots_code/model_state_dict')

gpath = f'{dpath}/gps.pkl'
gps_pd = pd.read_pickle(gpath)
train_pd = gps_pd[:1000]
test_pd = gps_pd[1000:2000].reset_index()

len_train = len(train_pd)
len_test = len(test_pd)
num_BS = 3

BATCH_SIZE = 64

if (user=='root'):
    n_worker = 2
else:
    n_worker = 8

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


## Utilities

In [14]:
def top_k_acc(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    
    y_pred_tpk = torch.topk(y_pred,k,dim=1)[1]
    
    ovr = 0
    pos = 0

    for i in range(0,len(y_pred_tpk)):
        if(y_true[i] in y_pred_tpk[i]):
            pos+=1
        ovr+=1
    
    acc = float(pos)/float(ovr)
    return acc

## Dataset

In [15]:
class bs_dataset(Dataset):
    def __init__(self,gps_pd:pd.DataFrame,
                lpath:str=dpath+'/lidar_samples',
                 label_path:str=dpath+'/labels'):
    
        self.gps = gps_pd
        self.lpath = lpath
        self.label_path = label_path
        
    def __getitem__(self,index):
        filename = self.gps.at[index,'Lidar']
        sample = dict() 

        sample['lidar'] = dict(np.load(os.path.join(self.lpath,filename)))['lidar'] #[10,240,240]
        sample['lidar'] = sample['lidar'].astype('float32')
        sample['gps'] = np.array(self.gps.at[index,'GPS'])[1].astype('float32') # Central GPS coord
        sample['BS'] = self.gps.at[index,'BS'].reshape((3*num_BS,)).astype('float32') #[num_BS*3,]
        sample['label'] = io.loadmat(self.label_path+f'/{filename[:-3]}mat')['ss']
        
        #Return the index of maximum element 
        sample['label'] = np.argmax(sample['label'].astype('float32')) 
        
        return sample
    
    def __len__(self):
        return len(self.gps)

In [16]:
train_dataset = bs_dataset(train_pd)
test_dataset = bs_dataset(test_pd)

## DataLoader

In [18]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    shuffle = True
    )

In [19]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True
)

# Model 

## Pre Trained Incp model

In [20]:
class incp(nn.Module):
    def __init__(self,drp:float=0.3,
                 drp_fc:float=0.2):
        super().__init__()
        
        self.drop_CNN = nn.Dropout(drp)
        self.drop_fc = nn.Dropout(drp_fc)
        self.mpool = nn.MaxPool2d((2,2))
        self.channels = 5
        self.fchannel = 3
        self.conv1 = self.create_conv(10,self.channels,13)
        self.conv2 = self.create_conv(self.channels,self.channels,13)
        self.conv3 = self.create_conv(self.channels,self.channels,7)
        self.conv4 = self.create_conv(self.channels,self.channels,7)

        # 1st Inception block
        self.in1 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in2 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in3 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.channels,self.fchannel,1)
        )
        self.in4 = self.create_conv(self.channels,self.fchannel,1)

        # Second inception block
        self.in5 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in6 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in7 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.fchannel*4,self.fchannel,1)
        )
        self.in8 = self.create_conv(self.fchannel*4,self.fchannel,1)

        self.conv5 = self.create_conv(self.fchannel*4,self.fchannel,7)
        self.conv6 = self.create_conv(self.fchannel,self.fchannel,7)

        self.conv7 = self.create_conv(self.fchannel,self.fchannel,3)
        self.conv8 = self.create_conv(self.fchannel,self.fchannel,3)
        
        self.flatten = nn.Flatten()

        # self.linear = nn.Sequential(
        #     nn.Linear(2352,256),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(256),
        #     self.drop_fc,
        #     nn.Linear(256,64),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(64),
        #     self.drop_fc,
        #     nn.Linear(64,16),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(16),
        #     self.drop_fc,
        #     nn.Linear(16,3)
        # )

        self.linear = nn.Sequential(
            nn.Linear(588,16),
            nn.PReLU(1),
            nn.BatchNorm1d(16),
            nn.Linear(16,3)
        )

    
    def forward(self,X):
        X  = self.conv1(X)
        X  = self.conv2(X)
        X = self.mpool(X)

        X = self.conv3(X)
        X = self.conv4(X)
        X = self.mpool(X)

        X = self.drop_CNN(X)

        a = self.in1(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in2(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in3(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in4(X)

        X = torch.cat((a,b,c,d),1)

        X = self.drop_CNN(X)

        a = self.in5(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in6(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in7(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in8(X)

        X = torch.cat((a,b,c,d),1)
        X = self.drop_CNN(X)

        X = self.conv5(X)
        X = self.conv6(X)

        X = self.mpool(X)

        X = self.drop_CNN(X)

        X = self.conv7(X)
        X = self.conv8(X)
        # print(X.shape)

        X = self.flatten(X)
        out = self.linear(X)

        return out
    
    def create_conv(self,in_layers:int,out_layers:int,kernel,stride:int=1,
                    padding:int=0) -> nn.Module :

        return nn.Sequential(
            nn.Conv2d(in_layers,out_layers,kernel,stride,padding),
            nn.PReLU(out_layers),
            nn.BatchNorm2d(out_layers)
        )

In [21]:
# lidar_pre = incp()

# if user == 'root':  #For google colab
#    lidar_pre.load_state_dict(torch.load('/gdrive/MyDrive/BTP/model_state_dict/lidar_final.zip'))
# else:
#     lidar_pre.load_state_dict(torch.load('./model_state_dict/lidar_final.zip'))

## Freezing layers
and forward hook

### Freezing

In [ ]:
# for c in lidar_pre.children():
#     if c == lidar_pre.linear:
# #         For freezing upto convolution, uncomment 'break' below
#         # break 
        
# #         for freezing upto last second layer
#         for params in lidar_pre.linear[0].parameters():
#             params.requires_grad = False

#     else:
#         for param in c.parameters():
#             param.requires_grad = False

In [ ]:
# for c in lidar_pre.conv8.parameters():
#     print(c.requires_grad)

# for c in lidar_pre.linear.parameters():
#     print(c.requires_grad)

### Forward Hook

In [ ]:
def getActivation(name,activation:dict):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

## Transfer Learning

In [ ]:
class incp_5(torch.nn.Module):
    def __init__(self,
                 lidar_pre:nn.Module=lidar_pre):
        
        super(incp_5,self).__init__()
        self.incp = lidar_pre
        
        self.activation = {}
        self.incp.flatten.register_forward_hook(getActivation('conv',self.activation))
        
        self.linear = nn.Sequential(
            nn.Linear(588,16),
            nn.PReLU(1),
            nn.BatchNorm1d(16),
            nn.Linear(16,5)
        )
        
    def forward(self,X):
        self.incp(X)
        out = self.activation['conv']
        
        out = self.linear(out)
        return out

In [ ]:
# model = incp_5(lidar_pre)

# Training

## Lidar Trainer


In [ ]:
class lidar_trainer(pl.LightningModule):
    def __init__(self,
                 drop_prob:float = 0.3,
                 drop_prob_fc:float = 0.2,
                 learning_rate:float = 1e-3,
                 weight_decay:float = 0.0):
        
        '''
        use_model = 'NU','imp'
        '''

        self.drp = drop_prob
        self.drp_fc = drop_prob_fc

        super().__init__()
        self.model = incp(drop_prob,drop_prob_fc)

        if user == 'root':  #For google colab
            self.model.load_state_dict(torch.load('/gdrive/MyDrive/BTP/model_state_dict/lidar_final.zip'))
        else:
            self.model.load_state_dict(torch.load('./model_state_dict/lidar_final.zip'))
        

        self.celoss = nn.CrossEntropyLoss()
        self.lr = learning_rate
        self.wd = weight_decay

        self.train_acc = 0
        self.val_acc = 0
        self.test_acc = 0

        self.example_input_array = torch.rand(1,10,240,240) # For logging graph
    
    def forward(self,lidar):
        out = self.model(lidar)
        return out
    
    def training_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self(lidar)
        
        loss = self.celoss(yhat,label)
        
        self.log('Train Loss',loss)
        
        return {'loss':loss,'pred':yhat.cpu().detach(),'label':label.cpu().detach()}
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(),self.lr,
                               weight_decay=self.wd)
        return opt
    
    def training_epoch_end(self,train_out):        
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)

        # Calculating Avg loss
        avg_loss = torch.stack([x['loss'] for x in train_out]).mean()
        
        self.logger.experiment.add_scalar('Loss-Train per epoch',avg_loss,self.current_epoch)
        self.logger.experiment.add_scalar('Train Accuracy',top1,self.current_epoch)

        self.train_acc = top1
        
        print(f'Train accuracies is {top1}')

    def validation_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        self.val_acc = top1

        self.logger.experiment.add_scalar('Validation Accuracy',top1,self.current_epoch)

        print(f'Validation accuracy is {top1}')
    
    def test_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def test_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        self.test_acc = top1

        self.logger.experiment.add_hparams(
            {
                'LR' : self.lr,
                'weight_decay' : self.wd,
                'drop_prob' : self.drp,
                'drop_fc' : self.drp_fc,
                'overall params' : sum(p.numel() for p in self.model.parameters())
            },
            {
                'hparam/test_acc' : self.test_acc,
                'hparam/train_acc' : self.train_acc,
                'hparam/val_acc' : self.val_acc
            }
        )
        print(f'Test accuracy is {top1}')

## Training

In [ ]:
lidar_model = lidar_trainer(learning_rate=1e-3)

In [ ]:
logger = TensorBoardLogger("tf_chicago", name="pre_trained",log_graph=True,default_hp_metric=False)
lidar_pl_trainer = pl.Trainer(
                     gpus=1,
                     max_epochs = 10,
                     precision = 16,
                     logger = logger,
                     log_every_n_steps = 1,
                     amp_backend = 'native',
                     deterministic = False,
                     auto_lr_find = False
                     )

In [ ]:
%tensorboard --logdir ./tf_5bs/init

In [ ]:
lidar_pl_trainer.tune(lidar_model,train_loader,test_loader)

In [ ]:
lidar_pl_trainer.fit(lidar_model,train_loader,test_loader)
lidar_pl_trainer.test(lidar_model,test_loader)

In [ ]:
!zip -r tf_5bs.zip tf_5bs